In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import quandl
import time
import datetime
import urllib3
# import scipy as sci
import scipy.stats as stats
import torch
import torch.nn as nn
import torch.nn.functional as F

from time import localtime, strftime

from multiprocessing import cpu_count
from multiprocessing import Pool
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import pdb
from pdb import set_trace as bp


In [14]:
start_date = "2003-01-21"
#end_date = time.strftime('%Y-%m-%d')
end_date = '2019-01-01'
authtoken="p3WBmpsMxN7sTWDoskqN"
print(end_date)

2019-01-01


In [78]:
#nasdaq = quandl.get("NASDAQOMX/NDX", authtoken=authtoken,
#                  start_date=start_date, end_date=end_date)
#nasdaq.to_pickle('nasdaq_us_df.pckl')
#nasdaq
nasdaq = pd.read_pickle('nasdaq_us_df.pckl')
nasdaq

,Index Value,High,Low,Total Market Value,Dividend Market Value
Trade Date,,,,,
2003-01-21,1008.93,1028.32,1008.93,1.460069e+09,0.000000e+00
2003-01-22,1027.08,1027.08,1003.58,1.487074e+09,3.916400e+04
2003-01-23,1039.16,1039.16,1011.12,1.504572e+09,0.000000e+00
2003-01-24,992.85,1027.70,992.85,1.437518e+09,0.000000e+00
2003-01-27,982.02,1006.26,982.02,1.421834e+09,0.000000e+00
2003-01-28,1006.51,1006.51,983.95,1.457291e+09,0.000000e+00
2003-01-29,1022.98,1022.98,980.92,1.481144e+09,0.000000e+00
2003-01-30,985.16,1022.21,985.16,1.426385e+09,7.372400e+04
2003-01-31,996.04,996.04,968.34,1.442129e+09,0.000000e+00


In [79]:
def get_percentage_change(data):
    N = len(data)
    final_array = []
    for index in range(N-1):
        final_array.append((data[index + 1] - data[index])/data[index])
    return np.array(final_array, dtype=np.float64)

In [80]:
nasdaq['index_value_change_pct'] = 100 * np.append(0,get_percentage_change(data=nasdaq['Index Value'].values))
nasdaq
nasdaq['index_value_change'] = np.append(0,get_percentage_change(data=nasdaq['Index Value'].values))
nasdaq

,Index Value,High,Low,Total Market Value,Dividend Market Value,index_value_change_pct,index_value_change
Trade Date,,,,,,,
2003-01-21,1008.93,1028.32,1008.93,1.460069e+09,0.000000e+00,0.000000,0.000000
2003-01-22,1027.08,1027.08,1003.58,1.487074e+09,3.916400e+04,1.798936,0.017989
2003-01-23,1039.16,1039.16,1011.12,1.504572e+09,0.000000e+00,1.176150,0.011761
2003-01-24,992.85,1027.70,992.85,1.437518e+09,0.000000e+00,-4.456484,-0.044565
2003-01-27,982.02,1006.26,982.02,1.421834e+09,0.000000e+00,-1.090799,-0.010908
2003-01-28,1006.51,1006.51,983.95,1.457291e+09,0.000000e+00,2.493839,0.024938
2003-01-29,1022.98,1022.98,980.92,1.481144e+09,0.000000e+00,1.636347,0.016363
2003-01-30,985.16,1022.21,985.16,1.426385e+09,7.372400e+04,-3.697042,-0.036970
2003-01-31,996.04,996.04,968.34,1.442129e+09,0.000000e+00,1.104389,0.011044


In [81]:

nasdaq['high_change_pct'] = 100 * np.append(0,get_percentage_change(data=nasdaq['High'].values))
nasdaq['high_change'] = np.append(0,get_percentage_change(data=nasdaq['High'].values))
nasdaq

/home/l7/dev/tools/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """


,Index Value,High,Low,Total Market Value,Dividend Market Value,index_value_change_pct,index_value_change,high_change_pct,high_change
Trade Date,,,,,,,,,
2003-01-21,1008.93,1028.32,1008.93,1.460069e+09,0.000000e+00,0.000000,0.000000,0.000000,0.000000
2003-01-22,1027.08,1027.08,1003.58,1.487074e+09,3.916400e+04,1.798936,0.017989,-0.120585,-0.001206
2003-01-23,1039.16,1039.16,1011.12,1.504572e+09,0.000000e+00,1.176150,0.011761,1.176150,0.011761
2003-01-24,992.85,1027.70,992.85,1.437518e+09,0.000000e+00,-4.456484,-0.044565,-1.102814,-0.011028
2003-01-27,982.02,1006.26,982.02,1.421834e+09,0.000000e+00,-1.090799,-0.010908,-2.086212,-0.020862
2003-01-28,1006.51,1006.51,983.95,1.457291e+09,0.000000e+00,2.493839,0.024938,0.024844,0.000248
2003-01-29,1022.98,1022.98,980.92,1.481144e+09,0.000000e+00,1.636347,0.016363,1.636347,0.016363
2003-01-30,985.16,1022.21,985.16,1.426385e+09,7.372400e+04,-3.697042,-0.036970,-0.075270,-0.000753
2003-01-31,996.04,996.04,968.34,1.442129e+09,0.000000e+00,1.104389,0.011044,-2.560139,-0.025601


In [82]:

nasdaq['low_change_pct'] = 100 * np.append(0,get_percentage_change(data=nasdaq['Low'].values))
nasdaq['low_change'] = np.append(0,get_percentage_change(data=nasdaq['Low'].values))
nasdaq

/home/l7/dev/tools/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """


,Index Value,High,Low,Total Market Value,Dividend Market Value,index_value_change_pct,index_value_change,high_change_pct,high_change,low_change_pct,low_change
Trade Date,,,,,,,,,,,
2003-01-21,1008.93,1028.32,1008.93,1.460069e+09,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2003-01-22,1027.08,1027.08,1003.58,1.487074e+09,3.916400e+04,1.798936,0.017989,-0.120585,-0.001206,-0.530265,-0.005303
2003-01-23,1039.16,1039.16,1011.12,1.504572e+09,0.000000e+00,1.176150,0.011761,1.176150,0.011761,0.751310,0.007513
2003-01-24,992.85,1027.70,992.85,1.437518e+09,0.000000e+00,-4.456484,-0.044565,-1.102814,-0.011028,-1.806907,-0.018069
2003-01-27,982.02,1006.26,982.02,1.421834e+09,0.000000e+00,-1.090799,-0.010908,-2.086212,-0.020862,-1.090799,-0.010908
2003-01-28,1006.51,1006.51,983.95,1.457291e+09,0.000000e+00,2.493839,0.024938,0.024844,0.000248,0.196534,0.001965
2003-01-29,1022.98,1022.98,980.92,1.481144e+09,0.000000e+00,1.636347,0.016363,1.636347,0.016363,-0.307942,-0.003079
2003-01-30,985.16,1022.21,985.16,1.426385e+09,7.372400e+04,-3.697042,-0.036970,-0.075270,-0.000753,0.432247,0.004322
2003-01-31,996.04,996.04,968.34,1.442129e+09,0.000000e+00,1.104389,0.011044,-2.560139,-0.025601,-1.707337,-0.017073


In [83]:

nasdaq['Total_Market_value_change_pct'] = 100 * np.append(0,get_percentage_change(data=nasdaq['Total Market Value'].values))
nasdaq['Total_Market_value_change'] = np.append(0,get_percentage_change(data=nasdaq['Total Market Value'].values))
nasdaq

,Index Value,High,Low,Total Market Value,Dividend Market Value,index_value_change_pct,index_value_change,high_change_pct,high_change,low_change_pct,low_change,Total_Market_value_change_pct,Total_Market_value_change
Trade Date,,,,,,,,,,,,,
2003-01-21,1008.93,1028.32,1008.93,1.460069e+09,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2003-01-22,1027.08,1027.08,1003.58,1.487074e+09,3.916400e+04,1.798936,0.017989,-0.120585,-0.001206,-0.530265,-0.005303,1.849586,0.018496
2003-01-23,1039.16,1039.16,1011.12,1.504572e+09,0.000000e+00,1.176150,0.011761,1.176150,0.011761,0.751310,0.007513,1.176686,0.011767
2003-01-24,992.85,1027.70,992.85,1.437518e+09,0.000000e+00,-4.456484,-0.044565,-1.102814,-0.011028,-1.806907,-0.018069,-4.456653,-0.044567
2003-01-27,982.02,1006.26,982.02,1.421834e+09,0.000000e+00,-1.090799,-0.010908,-2.086212,-0.020862,-1.090799,-0.010908,-1.091041,-0.010910
2003-01-28,1006.51,1006.51,983.95,1.457291e+09,0.000000e+00,2.493839,0.024938,0.024844,0.000248,0.196534,0.001965,2.493741,0.024937
2003-01-29,1022.98,1022.98,980.92,1.481144e+09,0.000000e+00,1.636347,0.016363,1.636347,0.016363,-0.307942,-0.003079,1.636777,0.016368
2003-01-30,985.16,1022.21,985.16,1.426385e+09,7.372400e+04,-3.697042,-0.036970,-0.075270,-0.000753,0.432247,0.004322,-3.697100,-0.036971
2003-01-31,996.04,996.04,968.34,1.442129e+09,0.000000e+00,1.104389,0.011044,-2.560139,-0.025601,-1.707337,-0.017073,1.103810,0.011038


In [84]:
def high_low_range(high, low, ref=None):
    if ref is None:
        return high - low
    else:
        return (high - low)/ ref

In [85]:

nasdaq['high_low_range'] = high_low_range(nasdaq['High'], nasdaq['Low'])
nasdaq['high_low_range_with_ref'] = high_low_range(nasdaq['High'], nasdaq['Low'], nasdaq['Index Value'])
nasdaq['high_low_range_with_ref_pct'] = 100 * high_low_range(nasdaq['High'], nasdaq['Low'], nasdaq['Index Value'])
nasdaq

,Index Value,High,Low,Total Market Value,Dividend Market Value,index_value_change_pct,index_value_change,high_change_pct,high_change,low_change_pct,low_change,Total_Market_value_change_pct,Total_Market_value_change,high_low_range,high_low_range_with_ref,high_low_range_with_ref_pct
Trade Date,,,,,,,,,,,,,,,,
2003-01-21,1008.93,1028.32,1008.93,1.460069e+09,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.39,0.019218,1.921838
2003-01-22,1027.08,1027.08,1003.58,1.487074e+09,3.916400e+04,1.798936,0.017989,-0.120585,-0.001206,-0.530265,-0.005303,1.849586,0.018496,23.50,0.022880,2.288040
2003-01-23,1039.16,1039.16,1011.12,1.504572e+09,0.000000e+00,1.176150,0.011761,1.176150,0.011761,0.751310,0.007513,1.176686,0.011767,28.04,0.026983,2.698333
2003-01-24,992.85,1027.70,992.85,1.437518e+09,0.000000e+00,-4.456484,-0.044565,-1.102814,-0.011028,-1.806907,-0.018069,-4.456653,-0.044567,34.85,0.035101,3.510097
2003-01-27,982.02,1006.26,982.02,1.421834e+09,0.000000e+00,-1.090799,-0.010908,-2.086212,-0.020862,-1.090799,-0.010908,-1.091041,-0.010910,24.24,0.024684,2.468381
2003-01-28,1006.51,1006.51,983.95,1.457291e+09,0.000000e+00,2.493839,0.024938,0.024844,0.000248,0.196534,0.001965,2.493741,0.024937,22.56,0.022414,2.241408
2003-01-29,1022.98,1022.98,980.92,1.481144e+09,0.000000e+00,1.636347,0.016363,1.636347,0.016363,-0.307942,-0.003079,1.636777,0.016368,42.06,0.041115,4.111517
2003-01-30,985.16,1022.21,985.16,1.426385e+09,7.372400e+04,-3.697042,-0.036970,-0.075270,-0.000753,0.432247,0.004322,-3.697100,-0.036971,37.05,0.037608,3.760810
2003-01-31,996.04,996.04,968.34,1.442129e+09,0.000000e+00,1.104389,0.011044,-2.560139,-0.025601,-1.707337,-0.017073,1.103810,0.011038,27.70,0.027810,2.781013


In [86]:
def binary_labeling(data, threshold):
    #bp()
    return data > threshold
    

In [92]:
nasdaq['gt_1'] = binary_labeling(nasdaq['index_value_change_pct'], 1)
nasdaq['gt_1.5'] = binary_labeling(nasdaq['index_value_change_pct'], 1.5)
nasdaq['gt_2.5'] = binary_labeling(nasdaq['index_value_change_pct'], 2.5)

nasdaq

,Index Value,High,Low,Total Market Value,Dividend Market Value,index_value_change_pct,index_value_change,high_change_pct,high_change,low_change_pct,low_change,Total_Market_value_change_pct,Total_Market_value_change,high_low_range,high_low_range_with_ref,high_low_range_with_ref_pct,gt_1.5,gt_2.5,gt_1
Trade Date,,,,,,,,,,,,,,,,,,,
2003-01-21,1008.93,1028.32,1008.93,1.460069e+09,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.39,0.019218,1.921838,False,False,False
2003-01-22,1027.08,1027.08,1003.58,1.487074e+09,3.916400e+04,1.798936,0.017989,-0.120585,-0.001206,-0.530265,-0.005303,1.849586,0.018496,23.50,0.022880,2.288040,True,False,True
2003-01-23,1039.16,1039.16,1011.12,1.504572e+09,0.000000e+00,1.176150,0.011761,1.176150,0.011761,0.751310,0.007513,1.176686,0.011767,28.04,0.026983,2.698333,False,False,True
2003-01-24,992.85,1027.70,992.85,1.437518e+09,0.000000e+00,-4.456484,-0.044565,-1.102814,-0.011028,-1.806907,-0.018069,-4.456653,-0.044567,34.85,0.035101,3.510097,False,False,False
2003-01-27,982.02,1006.26,982.02,1.421834e+09,0.000000e+00,-1.090799,-0.010908,-2.086212,-0.020862,-1.090799,-0.010908,-1.091041,-0.010910,24.24,0.024684,2.468381,False,False,False
2003-01-28,1006.51,1006.51,983.95,1.457291e+09,0.000000e+00,2.493839,0.024938,0.024844,0.000248,0.196534,0.001965,2.493741,0.024937,22.56,0.022414,2.241408,True,False,True
2003-01-29,1022.98,1022.98,980.92,1.481144e+09,0.000000e+00,1.636347,0.016363,1.636347,0.016363,-0.307942,-0.003079,1.636777,0.016368,42.06,0.041115,4.111517,True,False,True
2003-01-30,985.16,1022.21,985.16,1.426385e+09,7.372400e+04,-3.697042,-0.036970,-0.075270,-0.000753,0.432247,0.004322,-3.697100,-0.036971,37.05,0.037608,3.760810,False,False,False
2003-01-31,996.04,996.04,968.34,1.442129e+09,0.000000e+00,1.104389,0.011044,-2.560139,-0.025601,-1.707337,-0.017073,1.103810,0.011038,27.70,0.027810,2.781013,False,False,True


In [94]:
print(nasdaq['gt_1'].sum() / nasdaq.index.size)
print(nasdaq['gt_1.5'].sum() / nasdaq.index.size)
print(nasdaq['gt_2.5'].sum() / nasdaq.index.size)

0.18034825870646767
0.09776119402985074
0.03159203980099502


0.03159203980099502